In [1]:
# TrackingOff_Stationary_3_ios_background_fetch

In [2]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

#iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
#              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a"), UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
#android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
#               UUID("86842c35-da28-32ed-a90e-2da6663c5c73"), UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73")]

In [3]:
#type: list of emission.storage.timeseries.builtin_timeseries.BuiltinTimeSeries
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

#time query specification
dt1 = pydt.datetime(2016, 6, 30, 0, 30) #datetime.datetime(2016, 6, 24, 22, 0)
dt2 = pydt.datetime(2016, 7, 1, 9, 0)

ts1 = int(dt1.strftime("%s"))
ts2 = int(dt2.strftime("%s"))

time_range = estt.TimeQuery("metadata.write_ts", ts1, ts2)

#BuiltinTimeSeries.get_data_df(key, time_query = None, geo_query = None, extra_query_list=None) 
#type: list of DataFrame objects (regarding state info)
iphone_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), iphone_ts)
android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), android_ts)

iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), iphone_ts)
android_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), android_ts)

In [4]:
def get_ranges_df(battery_df, state_df):
    #find start_state_ts & end_state_ts for the period when state=3=tracking_stopped
    tracking_stopped_df = state_df[state_df.curr_state == 3]
    tracking_stopped_df.reset_index(inplace=True)
    start_state_ts = tracking_stopped_df.iloc[0].ts
    #if len(tracking_stopped_df)==1:
    #    end_state_ts = battery_df.iloc[-1].ts
    #else:
    #    end_state_ts = tracking_stopped_df.iloc[-1].ts
    end_state_ts = battery_df.iloc[-1].ts
    
    discharging_df = battery_df[(battery_df.ts >= start_state_ts) & (battery_df.ts <= end_state_ts) 
                                  & ((battery_df.battery_status == 1) )]
    discharging_df.reset_index(inplace=True)
    start_ts = discharging_df.iloc[0].ts
    end_ts = discharging_df.iloc[-1].ts
    
    ranges_df = pd.DataFrame({'start_ts': [start_ts], 'end_ts': [end_ts]})
    return ranges_df

In [5]:
def get_ranges_df_list(battery_df_list, state_df_list):
    ranges_df_list = []
    length = len(battery_df_list)
    for i in range(0, length):
        ranges_df_list.append(get_ranges_df(battery_df_list[i], state_df_list[i]))
    return ranges_df_list

In [6]:
iphone_ranges_df_list = get_ranges_df_list(iphone_battery_df_list, iphone_state_df_list)

iphone_concat_ranges_df = pd.concat([iphone_ranges_df_list[0], iphone_ranges_df_list[1], iphone_ranges_df_list[2]])
#concat_ranges_df = pd.concat([ranges_df_list[0], ranges_df_list[1], ranges_df_list[2], ranges_df_list[3]])
iphone_concat_ranges_df

,end_ts,start_ts
0,1.467365e+09,1.467262e+09
0,1.467366e+09,1.467261e+09
0,1.467347e+09,1.467347e+09


In [7]:
android_ranges_df_list = get_ranges_df_list(android_battery_df_list, android_state_df_list)

android_concat_ranges_df = pd.concat([android_ranges_df_list[0], android_ranges_df_list[1], android_ranges_df_list[2]])
#concat_ranges_df = pd.concat([ranges_df_list[0], ranges_df_list[1], ranges_df_list[2], ranges_df_list[3]])
android_concat_ranges_df

,end_ts,start_ts
0,1.467370e+09,1.467275e+09
0,1.467370e+09,1.467276e+09
0,1.467369e+09,1.467278e+09


In [9]:
iphone_start_ts = iphone_concat_ranges_df.start_ts.max() 
iphone_end_ts = iphone_concat_ranges_df.end_ts.min() 
print 'iphone_start_ts=' + str(iphone_start_ts) + '\n' + 'iphone_end_ts=' + str(iphone_end_ts)

android_start_ts = android_concat_ranges_df.start_ts.max() 
android_end_ts = android_concat_ranges_df.end_ts.min() 
print 'android_start_ts=' + str(android_start_ts) + '\n' + 'android_end_ts=' + str(android_end_ts)

iphone_start_ts=1467347372.69
iphone_end_ts=1467347372.69
android_start_ts=1467277750.69
android_end_ts=1467368582.97


In [10]:
import numpy as np

iphone_xnew = np.arange(iphone_start_ts, iphone_end_ts, 15 * 60) # 15 minutes
print len(iphone_xnew)

android_xnew = np.arange(android_start_ts, android_end_ts, 15 * 60) # 15 minutes
print type(android_xnew)

0
<type 'numpy.ndarray'>


In [11]:
import scipy.interpolate as spi
import numpy as np

def get_interpolated(x, y, xnew):
    # fn = spi.interp1d(x=x, y=y, bounds_error=False, fill_value=y.iloc[-1])
    fn = spi.interp1d(x=x, y=y, bounds_error=False)
    #This class returns a function whose call method uses interpolation to find the value of new points.
    return fn(xnew)

In [12]:
#android4: read from battery_stats
battery_df = pd.read_csv("../battery_script/batterystats4_fri_7_1_10am.csv", error_bad_lines=False)
ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
#battery_df['local_dt'] = battery_df['ts'].apply(ts_to_dt)
android_battery_df_list.append(battery_df)
battery_df.ts
get_interpolated(battery_df.ts, battery_df.battery_level_pct, android_xnew)
#print battery_df
#android4 = get_interpolated(battery_df.ts, battery_df.battery_level_pct, android_xnew)
#print android4

IOError: File ../battery_script/batterystats4_fri_7_1_10am.csv does not exist

In [ ]:
def get_interpolated_df(sensor_df, xnew):
    local_dt_new = map(pydt.datetime.fromtimestamp, xnew)
    fmt_time_new = map(lambda dt: dt.isoformat(), local_dt_new)
    #date.isoformat() Return a string representing the date in ISO 8601 format, ‘YYYY-MM-DD’. 
    # -For example, date(2002, 12, 4).isoformat() == '2002-12-04'.
    interpolated_series = get_interpolated(sensor_df.ts, sensor_df.battery_level_pct, xnew)
    # print "interpolated_series = %s" % interpolated_series
    interpolated_df = pd.DataFrame(data={'ts': xnew, 'metadata_write_ts': xnew, 'local_dt': local_dt_new, 'fmt_time': fmt_time_new, 'battery_level_pct': interpolated_series})
    return interpolated_df

In [13]:
iphone_battery_interp_df_list = map(lambda df: get_interpolated_df(df, iphone_xnew), iphone_battery_df_list)
android_battery_interp_df_list = map(lambda df: get_interpolated_df(df, android_xnew), android_battery_df_list)

NameError: global name 'get_interpolated_df' is not defined

In [14]:
label = ['10m', '30m', '1h', 'None']

In [ ]:
#iOS: No interplation on battery data
(afig, aaxes2d) = plt.subplots(ncols=1, nrows=3, sharex=True, sharey=True, figsize=(16,6))
aaxes = aaxes2d.flatten()

#afig.suptitle('iOS background Fetch', fontsize=14, fontweight='bold')
color = ['ob', 'og', 'or', 'oc']
c = ['b', 'g', 'r', 'c']

dt1 = pydt.datetime(2016, 6, 30, 0, 30) #datetime.datetime(2016, 6, 24, 22, 0)
dt2 = pydt.datetime(2016, 7, 1, 9, 0)

for i, df in enumerate(iphone_battery_df_list):
    print "About to plot %d" % i
    df['dt'] = df.ts.apply(ts_to_dt)
    df['hr'] = (df.ts-iphone_start_ts)/3600.0
    df.plot(x='dt', y='battery_level_pct', ylim=(92, 102), xlim=(dt1, dt2), style=color[i], ax=aaxes[i], label="iPhone%s " % (i+1) + '('+label[i]+')')
    #df.plot(x='ts', y='battery_level_pct', kind='scatter', ax=aaxes[i], title="iphone.%s" % (i+1), label=label[i], color=c[i])
    aaxes[i].set_xlabel('time')
    aaxes[i].set_ylabel('battery_level')

afig.savefig('plots/ios_background_fetch.pdf')

About to plot 0


NameError: name 'ts_to_dt' is not defined

In [ ]:
#iOS: With interplation on battery data 
'''
(ifig, iaxes2d) = plt.subplots(ncols=1, nrows=1, figsize=(16,6))
iaxes = iaxes2d
ifig.suptitle('No Tracking + Stationary_Jun_30-1 (iOS_interpolated_background_fetch)', fontsize=14, fontweight='bold')

for i, df in enumerate(iphone_battery_interp_df_list):
    print "About to plot %d" % i
    df.plot(x='local_dt', y='battery_level_pct', ax=iaxes, label=("iphone-%s " % (i+1))+'('+label[i]+')')
    #df.plot(x='ts', y='battery_level_pct', kind='scatter', ax=iaxes, label="iOS-%s" % i, color=color[i])
    #iaxes.set_xlabel('time')
    iaxes.set_ylabel('batery_level')
'''
#ifig.savefig('interp_Jun_30-1_iOS.png')    

In [ ]:
'''
(ifig, iaxes2d) = plt.subplots(ncols=1, nrows=1, figsize=(16,6))
iaxes = iaxes2d
color = ['ob', 'og', 'or', 'oc']
ifig.suptitle('No Tracking + Stationary_Jun_30-1 (iOS_overlaid)', fontsize=14, fontweight='bold')

for i, df in enumerate(iphone_battery_interp_df_list):
    print "About to plot %d" % i
    df.plot(x='local_dt', y='battery_level_pct', ax=iaxes, label=("iphone-%s " % (i+1))+'('+label[i]+')', legend=True)
    #df.plot(x='ts', y='battery_level_pct', kind='scatter', ax=iaxes, label="iOS-%s" % i, color=color[i])
    #iaxes.set_xlabel('time')
    iaxes.set_ylabel('batery_level')

for i, df in enumerate(iphone_battery_df_list):
    print "About to plot %d" % i
    df['dt'] = df.ts.apply(ts_to_dt)
    df.plot(x='dt', y='battery_level_pct', style=color[i], ax=iaxes, legend=False)
    #df.plot(x='ts', y='battery_level_pct', kind='scatter', ax=aaxes[i], title="iphone.%s" % (i+1), label="iphone-%s" % (i+1), color=color[i])
    
#ifig.savefig('overlaid_Jun_30-1_iOS.png')  
'''

In [ ]:
#iOS
#correlation analysis (tracking_off+stationary)
#generate list of iphone battery series 
iphone_battery_series_list = map(lambda b: b.battery_level_pct, iphone_battery_interp_df_list)
print np.corrcoef(iphone_battery_series_list)
plt.matshow(np.corrcoef(iphone_battery_series_list), cmap=plt.cm.gray)
plt.colorbar()